# Problem Set 4: Simulated Asymmetric Cryptography Environment

This notebook demonstrates a complete mini‑ecosystem with three actors:

* **Alice** – the sender who encrypts a secret message for Bob.
* **Bob** – the receiver who generates an RSA public/private key pair and decrypts Alice's message.
* **Eve** – an eavesdropper who tries to recover the plaintext by attacking RSA via integer factorisation.



##  Key generation for Bob

In [1]:
import sympy, math, random
bits = 128
e = 65537 # fix value (higher value of e = more secure algorithm)
while True:
    p = sympy.randprime(2**(bits-1), 2**bits) # first prime
    q = sympy.nextprime(p + random.randrange(1, 2**18)) # second prime, close to first just for easy calculation
    if p != q and math.gcd(e, (p-1)*(q-1)) == 1: # ensure p and q are different and greatest common divisor – największy wspólny dzielnik
        break
n = p*q
phi = (p-1)*(q-1)
d = pow(e, -1, phi)

## Alice encrypts a message

In [2]:
msg = "Hello Bob!"
m = int.from_bytes(msg.encode(), "big")
c = pow(m, e, n)
print(c)

15073030572527895729574345061834079640846642642090679248842159905584454500094


## Bob decrypts

In [8]:
plain = pow(c, d, n)
print(plain.to_bytes((plain.bit_length()+7)//8, "big").decode())

Hello Bob!


## Fermat Attack by Eve

In [3]:
import math, time
def fermat(n):
    a = math.isqrt(n) # start with the integer square root of n
    if a*a < n: # if a*a is less than n, we need to increment a
        a += 1
    while True:
        b2 = a*a - n
        b = math.isqrt(b2)
        if b*b == b2: # p = a-b, q = a+b
            return a-b, a+b
        a += 1

t = time.perf_counter()
pf, qf = fermat(n)
phi_f = (pf-1)*(qf-1)
d_f = pow(e, -1, phi_f)
m_eve = pow(c, d_f, n)
elapsed = time.perf_counter() - t
print(m_eve.to_bytes((m_eve.bit_length()+7)//8, "big").decode())
print("factorisation seconds:", round(elapsed,6))

Hello Bob!
factorisation seconds: 0.000225


$$  $$

We can describe n as:
$$ n = a^2 - b^2 $$
$$ n = (a-b)(a+b) $$

If p and q primes are close enough, we say that $$ (p+q)/2 $$ is approximately equal to $$ \sqrt{n} $$ Set  


$$ a = \bigl\lceil\sqrt{n}\bigr\rceil $$


(the first integer *not smaller* than $$(\sqrt n)$$).  
Because \(a\) is almost the average of \(p\) and \(q\), there exists a *small* integer \(b\) such that  


$$ n = a^{2} - b^{2} 
\;=\; (a-b)\,(a+b), $$


and therefore  

$$
p = a - b, 
\qquad
q = a + b .
$$
